In [1]:
import os 
import sys
sys.path.append("..")

import pickle 
import numpy as np
import torch 
import wandb

import tomondt
from tomondt import Operator, Viewer
from dipster.solver import Solver
from dipster import alignments, tomo

In [2]:
times=  np.linspace(1, 100, 100)
gr, alpha = (1+np.sqrt(5))/2, np.radians(140)
angles = np.degrees(((times *gr*alpha)%alpha)-(alpha/2))

ts_path = os.path.join('Y:\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries',"Cage-0-ts.pkl")
v_path = os.path.join('Y:\TimC\DIPSTER-PublicationData\Publication-Data\Volumes',"Cage-0.vmf")
ts = pickle.load(open(ts_path, 'rb'))
vmf = tomondt.load_vmf(v_path)

nc = vmf.read_record(0)
ts = tomondt.algorithms.forwardproject(vmf.read_record(0), angles)
ts = tomondt.TiltSeriesnDt(ts,angles, times)
#obj = tomondt.algorithms.sirt(ts)

num_rows = len(angles)
arr = np.tile(angles, (num_rows, 1)) 

batch = 4
sino = np.zeros((512, 512, 100,1))
obj =  np.zeros((512, 512, 512,1))
for i in range(512//batch):
    bs, be = batch*i, (batch*(i+1))
    sino[:,bs:be,:,:] = tomo.fp(vmf.read_record(0)[:,bs:be,:,None], angles)
    obj[:,bs:be,:,:] = tomo.bp(sino[:,bs:be,:,:], arr, iters=10)
sino = sino.squeeze()
#obj = obj.squeeze()


import copy
obj2 = copy.deepcopy(obj)

In [3]:

#obj = tomondt.utils.permute(obj2, (2,1,0,3)).squeeze()

viewer = tomondt.Viewer()

viewer.add_image(nc[256,:,:].squeeze(), name='x ref')
viewer.add_image(obj[256,:,:].squeeze(), name='x obj')

viewer.add_image(nc[:,256,:].squeeze(), name='y ref')
viewer.add_image(obj[:,256,:].squeeze(), name='y obj')

viewer.add_image(nc[:,:,256].squeeze(), name='z ref')
viewer.add_image(obj[:,:,256].squeeze(), name='z obj')
#list = np.argsort(angles)





<Image layer 'z obj' at 0x1c2c4de4110>